In [5]:
# Task 1. Extraction of .7z zip files
import os
import shutil
import py7zr
from pathlib import Path

def extract_7z_file(zip_path, output_folder, password):
    with py7zr.SevenZipFile(zip_path, mode='r', password=password) as archive:
        archive.extractall(output_folder)

def extract_and_organize(zip_folders, output_base_folder, password):
    output_base_folder = Path(output_base_folder)

    for zip_folder_path in zip_folders:
        zip_folder = Path(zip_folder_path)

        # Ensure the output base folder exists
        output_base_folder.mkdir(parents=True, exist_ok=True)

        # Process each .7z file in the zip folder
        for zip_file in zip_folder.glob('*.7z'):
            # Extract folder name from the .7z file (excluding the extension)
            folder_name = zip_file.stem

            # Create an output folder based on the extracted folder name
            output_folder = output_base_folder / folder_name
            output_folder.mkdir(parents=True, exist_ok=True)

            # Extract the .7z file to the corresponding output folder
            extract_7z_file(zip_file, output_folder, password)

# Set the paths to the folders containing .7z files
zip_folders_paths = ["C:/Users/Praful Bhoyar/Desktop/Sam Project"]
# Set the path to the base folder where extracted files will be organized
output_base_folder_path = "C:/Users/Praful Bhoyar/Desktop/extracted7z"

# Set the common password for all .7z files
common_password = "6604667cd6f4785f728bbcefa0979ddde53ad134f06191ec0525990d153bcbe5"

# Extract and organize the files from .7z folders
extract_and_organize(zip_folders_paths, output_base_folder_path, common_password)


In [6]:
# Task 2: Extract the nested files
import os
import zipfile

def extract_zip_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.zip'):
                zip_path = os.path.join(root, file)
                extract_path = os.path.splitext(zip_path)[0]  # Remove the .zip extension
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_path)
                extract_zip_files(extract_path)  # Recursively extract nested zip files

if __name__ == "__main__":
    input_directory = "C:/Users/Praful Bhoyar/Desktop/extracted7z"
    extract_zip_files(input_directory)


In [7]:
# Task 3 Setup the dataframe of missing values dataframe
import pandas as pd
df_missing_data = pd.read_excel("C:/Users/Praful Bhoyar/Desktop/missing excel/s3 missing files.xlsx")
df_missing_data.head()

,directory,filename,status,SharepointPath,Zipfile,S3_size,S3_unit,S3_dttm
0,00532A5F2F9B46039C7E,00532A5F2F9B46039C7E_473F327E359732989D991500F...,not in S3,Lizzy's Downloads pages 1 to 12,extract_images_p0086903ui5v_20230706203430-1@1...,NaN,NaN,
1,000746ED3DF24C09AA1B,000746ED3DF24C09AA1B_512283C08D4F3A6281357A43B...,not in S3,Lizzy's Downloads pages 1 to 12,extract_images_p0086903ui5v_20230706203430-1@1...,NaN,NaN,
2,,002820D90BD049028CAC.zip,not in S3,Lizzy's Downloads pages 1 to 12,extract_images_p0086903ui5v_20230706203430-1@1...,NaN,NaN,
3,002AF2DF5F2A444E97C5,002AF2DF5F2A444E97C5_53880754D03B313D84609CC22...,not in S3,Lizzy's Downloads pages 1 to 12,extract_images_p0086903ui5v_20230706203430-1@1...,NaN,NaN,
4,004ED2DC636C4886BD30,004ED2DC636C4886BD30_3F9ED562770038E0AEC1E71C4...,not in S3,Lizzy's Downloads pages 1 to 12,extract_images_p0086903ui5v_20230706203430-1@1...,NaN,NaN,


In [8]:
import os
import shutil
import pandas as pd

def copy_files_based_on_dataframe(df, source_folder, destination_base_folder, exclude_extensions=None):
    not_found_files = {}  # Dictionary to store files not found in each folder

    for index, row in df.iterrows():
        filename = row['filename']
        found = False  # Flag to check if the file is found in any folder

        for folder_name in os.listdir(source_folder):
            folder_path = os.path.join(source_folder, folder_name)
            file_path = find_file_recursive(folder_path, filename)

            if file_path and not has_extension(file_path, exclude_extensions):
                destination_base_folder = destination_base_folder or os.path.join(os.path.expanduser("~"), "Desktop")
                missing_folder_path = os.path.join(destination_base_folder, f"missing_files_{folder_name}")

                # Create 'missing_files' folder for each zip folder
                os.makedirs(missing_folder_path, exist_ok=True)

                destination_path = os.path.join(missing_folder_path, filename)

                # Check for duplicates before copying
                if not os.path.exists(destination_path):
                    # Copy the file to the destination folder
                    shutil.copy(file_path, destination_path)
                    found = True
                    print(f"Successfully copied {filename} to {destination_path} from {folder_name}")
                else:
                    print(f"File {filename} already exists in the destination folder from {folder_name}")

        if not found:
            not_found_files[filename] = "Not found in any folder"

    return not_found_files

def find_file_recursive(search_folder, target_filename):
    for root, dirs, files in os.walk(search_folder):
        if target_filename in files:
            return os.path.join(root, target_filename)
    return None

def has_extension(file_path, extensions):
    if extensions is None:
        return False
    _, file_extension = os.path.splitext(file_path)
    return file_extension.lower() in extensions

# Example DataFrame with filenames in the "filename" column
df_miss_data = df_missing_data

# Set the path to the folder containing the files
source_folder_path = output_base_folder_path

# Specify the extensions to exclude (e.g., ['.zip'])
exclude_extensions = []

# Copy files based on DataFrame information, excluding specified extensions
not_found_files = copy_files_based_on_dataframe(df_miss_data, source_folder_path, None, exclude_extensions)

# Display the list of files not found in any folder
print("Files not found in any folder:")
print(not_found_files)

Successfully copied 00532A5F2F9B46039C7E_473F327E359732989D991500F68B2E9B.jpg to C:\Users\Praful Bhoyar\Desktop\missing_files_extract_images_p0086903ui5v_20230706203430-1@1_1of1.zip\00532A5F2F9B46039C7E_473F327E359732989D991500F68B2E9B.jpg from extract_images_p0086903ui5v_20230706203430-1@1_1of1.zip
Successfully copied 000746ED3DF24C09AA1B_512283C08D4F3A6281357A43B308480A.pdf to C:\Users\Praful Bhoyar\Desktop\missing_files_extract_images_p0086903ui5v_20230706203430-1@1_1of1.zip\000746ED3DF24C09AA1B_512283C08D4F3A6281357A43B308480A.pdf from extract_images_p0086903ui5v_20230706203430-1@1_1of1.zip
Successfully copied 002820D90BD049028CAC.zip to C:\Users\Praful Bhoyar\Desktop\missing_files_extract_images_p0086903ui5v_20230706203430-1@1_1of1.zip\002820D90BD049028CAC.zip from extract_images_p0086903ui5v_20230706203430-1@1_1of1.zip
Successfully copied 002AF2DF5F2A444E97C5_53880754D03B313D84609CC223F69919.jpg to C:\Users\Praful Bhoyar\Desktop\missing_files_extract_images_p0086903ui5v_202307062

In [9]:
not_found_files

{}